# Kryptografia postkwantowa oparta o funkcje skrótu
Magdalena Ślusarczyk, Michał Bełzak - Cyberbezpieczeństwo 21/22  

Do działania wymagany jest python w wersji >=3.10 lub ręczne usunięcie wszystkich opisów typów z notebooka.
## Systemy podpisów jednorazowych

### System Lamporta


In [1]:
import hashlib
import secrets

def lamportFromBytesToBinary(bytesToConvert: bytes) -> str:
    '''Converts bytes to binary bits string.'''
    binaryList: list[bytes] = []
    for byte in bytesToConvert:
        binaryList.append(bin(byte)[2:].zfill(8))
    return ''.join(binaryList)

def lamportGenerateKeyPair() -> tuple[list[list[bytes]], list[list[bytes]]]:
    '''Generates (secretKey, publicKey) pair, where secretKey is secret and publicKey is public key.'''
    secretKey: list[list[bytes]] = []
    publicKey: list[list[bytes]] = []
    for i in range(256): # generates 256 pairs of random bytes for secret key and hashes each of them to create public key
        secretKey.append([secrets.token_bytes(32), secrets.token_bytes(32)])
        publicKey.append([hashlib.sha256(secretKey[i][0]).digest(), hashlib.sha256(secretKey[i][1]).digest()])
    return (secretKey, publicKey)

def lamportSign(message: str, secretKey: list[list[bytes]]) -> list[bytes]:
    '''Generates signature for the message.'''
    signature: list[bytes] = []
    messageHash = lamportFromBytesToBinary(hashlib.sha256(message.encode('utf-8')).digest()) # hashes the message and converts the hash from bytes to binary bits
    for i in range(256): # creates signature by choosing first or second value from secret key, depending on the value of corresponding bit in message hash (for every pair)
        signature.append(secretKey[i][int(messageHash[i])])
    return signature

def lamportVerifySignature(signature: list[bytes], message: str, publicKey: list[list[bytes]]) -> bool: 
    '''Returns True if signature is compatible with public key, False otherwise.'''
    messageHash = lamportFromBytesToBinary(hashlib.sha256(message.encode('utf-8')).digest()) # again hashes the message and converts the hash from bytes to binary bits
    for i in range(len(signature)): # hashes every element in signature and compares it with corresponding value in public key 
        elementHash = hashlib.sha256(signature[i]).digest()
        if elementHash != publicKey[i][int(messageHash[i])]:
            return False
    return True


keypair = lamportGenerateKeyPair()
message = "This is message"
signature = lamportSign(message, keypair[0])
print(f"Signature verification: {lamportVerifySignature(signature, message, keypair[1])}")

Signature verification: True


### System Winternitza

In [2]:
def wFromBytesToBinary(bytes: bytes) -> list[str]:
    '''Converts bytes to list of 8bit binary strings.'''
    binaryList: list[str] = []
    for byte in bytes:
        binaryList.append(bin(byte)[2:].zfill(8))
    return binaryList


def winterGenerateKeyPair() -> tuple[list[bytes], list[bytes]]:
    '''Generates (secretKey, publicKey) pair, where secretKey is secret and publicKey is public key.'''
    secretKey: list[bytes] = []
    publicKey: list[bytes] = []
    for i in range(32): # generates 32 random values for secret key and hashes every one of them 256 times to create public key
        secretKey.append(secrets.token_bytes(32))
        publicKeyElement = hashlib.sha256(secretKey[i]).digest()
        for j in range(1, 256):
            publicKeyElement = hashlib.sha256(publicKeyElement).digest()
        publicKey.append(publicKeyElement)
    return (secretKey, publicKey)

def winterSign(message: str, secretKey: list[bytes]) -> list[bytes]:
    '''Generates signature for the message.'''
    signature: list[bytes] = []
    messageHash = wFromBytesToBinary(hashlib.sha256(message.encode('utf-8')).digest()) # hashes the message and converts it to 8-bit binary strings
    for i in range(len(messageHash)): 
        N = int(messageHash[i], 2) # converts value N (8-bit string from message hash) from binary to decimal
        signatureElement = hashlib.sha256(secretKey[i]).digest() 
        for j in range(1, 256 - N): # creates the signature by hashing every element of secret key 256-N times
            signatureElement = hashlib.sha256(signatureElement).digest()
        signature.append(signatureElement)
    return signature

def winterVerifySignature(signature: list[bytes], message: str, publicKey: list[bytes]) -> bool: 
    '''Returns True if signature is compatible with public key, False otherwise.'''
    messageHash = wFromBytesToBinary(hashlib.sha256(message.encode('utf-8')).digest())
    for i in range(len(signature)):
        N = int(messageHash[i], 2)
        signatureElement = hashlib.sha256(signature[i]).digest()
        for j in range(1, N): # hashes every element in signature another N times and compares it with corresponding value in public key
            signatureElement = hashlib.sha256(signatureElement).digest()
        if signatureElement != publicKey[i]:
            return False
    return True

keypair = winterGenerateKeyPair()
message = "This is message"
signature = winterSign(message, keypair[0])
print(f"Signature verification: {winterVerifySignature(signature, message, keypair[1])}")

Signature verification: True


## Drzewo Merkla

In [3]:
class MerkleTree:
    messageCount: int = 0
    merklePrivateKey: list[list[bytes]] = []
    merklePublicKey: bytes = []
    lamportKeypairs: list[tuple[list[bytes], list[bytes]]] = []
    nodes: list[bytes] = []
    height: int
    def __init__(self, treeHeight: int = 3) -> None:
        """Generate Merkle Tree with OTS Lamport keys"""
        self.height = treeHeight
        for el in range(2**(treeHeight+1)-1): #create empty nodes
            self.nodes.append(b'')
        
        for keypairOffset in range(2**treeHeight): #generate hashes of public keys in leaves
            keypair = lamportGenerateKeyPair()
            self.lamportKeypairs.append(keypair)
            joined = [b''.join(el) for el in keypair[1]]
            joined = b''.join(joined)
            self.nodes[2**treeHeight + keypairOffset - 1] = hashlib.sha256(joined).digest()
            self.merklePrivateKey.append(keypair[0])


        for node in reversed(range(2**treeHeight - 1)):
            self.nodes[node] = hashlib.sha256(self.nodes[2*(node+1)-1] + self.nodes[2*(node+1)]).digest()

        self.merklePublicKey = self.nodes[0]
        return
        
    def getMerkleKeys(self):
        return (self.merklePrivateKey, self.merklePublicKey)

    def signMessage(self, message):
        if self.messageCount == 2**self.height:
            print(f'Error, keys exhausted. Instantiate a new tree!')
            return
        signature = lamportSign(message, self.merklePrivateKey[self.messageCount])
        

        path = []
        curr_node = 2**self.height + self.messageCount
        for h in range(self.height):
            if curr_node % 2 == 0:
                path.append(self.nodes[curr_node])
            else:
                path.append(self.nodes[curr_node-2])
            curr_node = curr_node//2

        self.messageCount += 1
        return (self.messageCount-1, signature, self.lamportKeypairs[self.messageCount-1][1], path)


def verifyMerkle(numberOfMessage, lamportSign, lamportPublicKey, verificationPath, message, merklePublicKey):
    #step 1 - verify signature with Lamport scheme
    if lamportVerifySignature(lamportSign,message,lamportPublicKey) != True:
        print(f'Lamport verify failed!')
        return False
    
    #step 2 - verify Lamport public key 
    joined = [b''.join(el) for el in lamportPublicKey]
    joined = b''.join(joined)
    keyDigest = hashlib.sha256(joined).digest()
    idx = numberOfMessage
    for node in verificationPath:
        if idx % 2 == 1:
            keyDigest = hashlib.sha256(node+keyDigest).digest()
        else:
            keyDigest = hashlib.sha256(keyDigest+node).digest()
        idx = idx// 2
    
    if keyDigest == merklePublicKey:
        return True
    return False


tree = MerkleTree(4)
merklePublic = tree.getMerkleKeys()[1]

print(f'Merkle public key: {merklePublic.hex()}')

message = "kokodzambo"
signed = tree.signMessage(message)
print(f'Message no. {signed[0]}')
print(f'First pair of Lamport signature {signed[1][0].hex()}')
print(f'Message verification path {[el.hex() for el in signed[3]]}')
print(f'Message signature verification status: {verifyMerkle(signed[0],signed[1],signed[2],signed[3],message,merklePublic)}')
print('-'*40)

signed = tree.signMessage(message)
print(f'Message no. {signed[0]}')
print(f'First pair of Lamport signature {signed[1][0].hex()}')
print(f'Message verification path {[el.hex() for el in signed[3]]}')
print(f'Message signature verification status: {verifyMerkle(signed[0],signed[1],signed[2],signed[3],message,merklePublic)}')
print('-'*40)
signed = tree.signMessage(message)
print(f'Message no. {signed[0]}')
print(f'First pair of Lamport signature {signed[1][0].hex()}')
print(f'Message verification path {[el.hex() for el in signed[3]]}')
print(f'Message signature verification status: {verifyMerkle(signed[0],signed[1],signed[2],signed[3],message,merklePublic)}')
print('-'*40)
signed = tree.signMessage(message)
print(f'Message no. {signed[0]}')
print(f'First pair of Lamport signature {signed[1][0].hex()}')
print(f'Message verification path {[el.hex() for el in signed[3]]}')
print(f'Message signature verification status: {verifyMerkle(signed[0],signed[1],signed[2],signed[3],message,merklePublic)}')
print('-'*40)

Merkle public key: 266ace4f2b155967a40ed77341394c635ba2c096ae21e4afaacf373350892d7b
Message no. 0
First pair of Lamport signature 6a9129f125e20ed238d8ef9df557e86c07840b7f2cfd3934af8a96584d284074
Message verification path ['8974a6890186c73584465b741b5deb1ffc4c3945794f2477f8e37ac902611250', '20529b6767694aa6bc0b35ea6bcf41f3b21c13dbcf584f82e70c7fb7508e3875', '7cb9be060488e598118d96d0b1dcb424c1e2998c8b8ba9c48df6bc257a9b6767', '3ade4f8a27d5a331d99ac94c5e6c45c8ec5f0096aaf18b282bcafe3efd4b5364']
Message signature verification status: True
----------------------------------------
Message no. 1
First pair of Lamport signature 3acf2fb4b750a3942f3dcfe7fb81618f547793ba8b2b1c930d8acfb7b98a74dd
Message verification path ['9fca0dc19b653c28f473b397c3c40341766375ae8b1fc074720aa0e3a9be4a38', '20529b6767694aa6bc0b35ea6bcf41f3b21c13dbcf584f82e70c7fb7508e3875', '7cb9be060488e598118d96d0b1dcb424c1e2998c8b8ba9c48df6bc257a9b6767', '3ade4f8a27d5a331d99ac94c5e6c45c8ec5f0096aaf18b282bcafe3efd4b5364']
Message sig